In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt

import ruptures as rpt
from scipy.stats import mannwhitneyu

In [2]:
exp1_path = '/Users/anzhunie/Desktop/Pedestrian_Training/Data/Experiment1.csv'
exp2_path = '/Users/anzhunie/Desktop/Pedestrian_Training/Data/Experiment2.csv'
exp3_path = '/Users/anzhunie/Desktop/Pedestrian_Training/Data/Experiment3.csv'

In [3]:
import pandas as pd
import numpy as np

def read_csv(file_path, num_exp):
    df = pd.read_csv(file_path)


    # Convert only the numeric columns to numeric values, handling invalid entries
    numeric_columns = ['Time', 'ID', 'Positionx', 'Positionz', 'Positiony', 'Yaw', 'Up', 'Right', 'Down', 'Left', 'Trajectory', 'Distance', 'Speed', 'Direction']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')  # Coerce invalid values to NaN

    # Helper function to format Trajectory_ID
    def format_trajectory(row):
        if pd.isna(row['ID']) or pd.isna(row['Trajectory']):
            return np.nan  # Keep NaN if ID or Trajectory is missing
        formatted_id = f"{int(row['ID']):03d}"  # Zero-padded ID
        formatted_trajectory = f"{int(row['Trajectory']):03d}"  # Zero-padded Trajectory
        formatted_exp_num = f"{int(num_exp)}"  # Experiment number
        return f"{formatted_exp_num}{formatted_id}{formatted_trajectory}"

    # Apply the formatting function to create the new column
    df['Trajectory_ID'] = df.apply(format_trajectory, axis=1)

    return df


In [4]:
def label_crowd_radius(df):

    crowd_radius = 2.58

    # Initialize 'Label' column as NaN (Only assign labels to rows with a valid Trajectory_ID)
    df['Crowd_Radius_Label'] = np.nan

    # Iterate over each unique Trajectory_ID (excluding NaNs)
    for trajectory_id in df['Trajectory_ID'].dropna().unique():
        # Extract the subset for this trajectory
        trajectory_section = df[df['Trajectory_ID'] == trajectory_id].copy()
        
        # Step 1: Label all 'Inside' points (Distance ≤ crowd_radius)
        df.loc[trajectory_section.index[trajectory_section['Distance'] <= crowd_radius], 'Crowd_Radius_Label'] = 'Inside'
        
        # Step 2: Label all 'Outside' points (Distance > crowd_radius)
        df.loc[trajectory_section.index[trajectory_section['Distance'] > crowd_radius], 'Crowd_Radius_Label'] = 'Outside'
        
        # Step 3: Find the last 'Outside' index for this trajectory
        last_outside_idx = (
            trajectory_section[trajectory_section['Distance'] > crowd_radius].index[-1] 
            if not trajectory_section[trajectory_section['Distance'] > crowd_radius].empty 
            else None
        )
        
        # Step 4: Change all 'Inside' points before the last 'Outside' to 'Inside-Out'
        if last_outside_idx is not None:
            # Apply both conditions together to avoid index mismatches
            inside_before_last_outside = trajectory_section.loc[
                (trajectory_section.index <= last_outside_idx) & (trajectory_section['Distance'] <= crowd_radius)
            ].index
            
            # Update labels for these points
            df.loc[inside_before_last_outside, 'Crowd_Radius_Label'] = 'Inside-Out'
    
    return df  # Return the modified DataFrame with labels


In [5]:
exp1 = label_crowd_radius(read_csv(exp1_path,1))
exp2 = label_crowd_radius(read_csv(exp2_path,2))
exp3 = label_crowd_radius(read_csv(exp3_path,3))

In [6]:
df_total = pd.concat([exp1, exp2, exp3], ignore_index=True)

In [8]:
# Count occurrences of each unique Trajectory_ID
identical_traj_counts = df_total['Trajectory_ID'].value_counts()

# Display how many times each Trajectory_ID appears
print(identical_traj_counts)

# Count how many Trajectory_IDs have duplicates
num_identical_trajs = (identical_traj_counts > 1).sum()

print(f"Number of duplicated Trajectory_IDs: {num_identical_trajs}")

Trajectory_ID
1071003    391
3146012    333
1078010    230
3182001    226
2253001    181
          ... 
1062021     10
2283018     10
1043030     10
3165020      9
1051001      9
Name: count, Length: 1300, dtype: int64
Number of duplicated Trajectory_IDs: 1300


In [7]:
dfs = [exp1, exp2, exp3]
df_total = pd.DataFrame()

#add change of speed and direction
for df in dfs:
    df['Speed Change'] = np.nan
    df['Direction Change'] = np.nan
    for i in range(df.shape[0]-1):
        if pd.notna(df.iloc[i+1]['Speed']):
                if pd.notna(df.iloc[i]['Speed']):
                    df.at[i,'Speed Change'] = df.iloc[i+1]['Speed'] - df.iloc[i]['Speed']
        if pd.notna(df.iloc[i+1]['Direction']):
            if pd.notna(df.iloc[i]['Direction']):
                diff = df.iloc[i+1]['Direction'] - df.iloc[i]['Direction']
                # Wrap the difference to the range -pi to pi
                if diff > np.pi:
                    diff -= 2 * np.pi
                elif diff < -np.pi:
                    diff += 2 * np.pi
                df.at[i, 'Direction Change'] = diff

    df_total = pd.concat([df_total, df], axis=0)


In [8]:
df_total

,Time,ID,Positionx,Positionz,Positiony,Yaw,Up,Right,Down,Left,Trajectory,Distance,Speed,Direction,Trajectory_ID,Crowd_Radius_Label,Speed Change,Direction Change
0,0.0,78.0,1.05,0.0,-0.95,-4.251929,0.0,0.0,0.0,1.0,1.0,1.42,NaN,NaN,1078001,Inside,NaN,NaN
1,0.5,78.0,1.06,0.0,-0.96,-3.775285,1.0,0.0,0.0,1.0,1.0,1.43,0.03,-0.79,1078001,Inside,0.01,-0.32
2,1.0,78.0,1.07,0.0,-0.98,-3.747693,1.0,0.0,0.0,1.0,1.0,1.45,0.04,-1.11,1078001,Inside,0.05,1.57
3,1.5,78.0,1.11,0.0,-0.96,-3.757570,1.0,0.0,0.0,1.0,1.0,1.47,0.09,0.46,1078001,Inside,-0.09,0.00
4,2.0,78.0,1.11,0.0,-0.96,-3.508991,1.0,0.0,0.0,1.0,1.0,1.47,0.00,0.46,1078001,Inside,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29541,669.0,170.0,-0.29,0.0,-0.94,1.393295,0.0,1.0,0.0,0.0,27.0,0.98,0.82,1.52,3170027,Inside,-0.29,-0.29
29542,669.5,170.0,-0.20,0.0,-0.69,1.377272,0.0,1.0,0.0,0.0,27.0,0.72,0.53,1.23,3170027,Inside,-0.51,0.34
29543,670.0,170.0,-0.20,0.0,-0.68,1.413267,0.0,1.0,0.0,0.0,27.0,0.71,0.02,1.57,3170027,Inside,0.21,-0.91
29544,670.5,170.0,-0.11,0.0,-0.61,1.400711,0.0,1.0,0.0,0.0,27.0,0.62,0.23,0.66,3170027,Inside,NaN,NaN


In [16]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from tslearn.metrics import dtw
from sklearn.manifold import MDS


In [12]:
# 过滤出 Crowd_Radius_Label 为 'Inside' 的行
df_inside = df_total[df_total['Crowd_Radius_Label'] == 'Inside']

# 选择 inside data 进行聚类
df_features_inside = df_inside[['Trajectory',
                                'Speed Change','Direction Change']].dropna()

# 对 inside data 的这些特征进行标准化
scaler = StandardScaler()
df_features_inside_scaled = pd.DataFrame(scaler.fit_transform(df_features_inside.iloc[:, 1:]),  # 忽略 'Trajectory' 列
                                         columns=df_features_inside.columns[1:],  # 保持特征名
                                         index=df_features_inside.index)

# 恢复 'Trajectory' 列
df_features_inside_scaled['Trajectory'] = df_features_inside['Trajectory']

In [13]:
# 将 inside data 按 'Trajectory' 分组，生成每个轨迹的特征序列
trajectories_inside = []
trajectory_ids_inside = df_features_inside_scaled['Trajectory'].unique()

for traj_id in trajectory_ids_inside:
    traj_data = df_features_inside_scaled[df_features_inside_scaled['Trajectory'] == traj_id][['Speed Change','Direction Change']].values
    trajectories_inside.append(np.asarray(traj_data)) 

# 计算所有轨迹对之间的DTW距离
n = len(trajectories_inside)
distance_matrix_inside = np.zeros((n, n))

for i in range(n):
    for j in range(i, n):
        distance = dtw(trajectories_inside[i], trajectories_inside[j])
        distance_matrix_inside[i, j] = distance
        distance_matrix_inside[j, i] = distance

# 使用 MDS 进行降维
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=0)
mds_transformed_inside = mds.fit_transform(distance_matrix_inside)

/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [14]:
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.metrics import pairwise_distances

# Assuming `mds_transformed_inside` is your MDS-transformed data
distance_matrix = pairwise_distances(mds_transformed_inside, metric='euclidean')

best_score = -1
best_k = 0
best_labels = None

# Try different numbers of clusters to find the optimal K
for k in range(2, 10):  # Trying from 2 to 10 clusters
    kmedoids = KMedoids(n_clusters=k, metric='precomputed', random_state=42)
    labels = kmedoids.fit_predict(distance_matrix)  # Fit to the precomputed distance matrix
    score = silhouette_score(mds_transformed_inside, labels)  # Compute silhouette score on MDS-transformed data

    print(f"Number of clusters: {k}, Silhouette Score: {score}")

    if score > best_score:
        best_score = score
        best_k = k
        best_labels = labels

print(f"Best number of clusters by silhouette score: {best_k}")


Number of clusters: 2, Silhouette Score: 0.4286368062557691
Number of clusters: 3, Silhouette Score: 0.4431508778729626
Number of clusters: 4, Silhouette Score: 0.3964429782059626
Number of clusters: 5, Silhouette Score: 0.31266239611390734
Number of clusters: 6, Silhouette Score: 0.2863802491819205
Number of clusters: 7, Silhouette Score: 0.3203838977260738
Number of clusters: 8, Silhouette Score: 0.28264944390737046
Number of clusters: 9, Silhouette Score: 0.2549306414530764
Best number of clusters by silhouette score: 3


In [15]:
best_score = -1
best_k = 0
best_labels = None

# 尝试不同的簇数，寻找最佳的 K
for k in range(2, 10):  # 从2到10个簇进行尝试
    kmeans = KMeans(n_clusters=k, random_state=0)
    labels = kmeans.fit_predict(mds_transformed_inside)
    score = silhouette_score(mds_transformed_inside, labels)

    print(f"Number of clusters: {k}, Silhouette Score: {score}")

    if score > best_score:
        best_score = score
        best_k = k
        best_labels = labels

print(f"Best number of clusters by silhouette score: {best_k}")


/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Number of clusters: 2, Silhouette Score: 0.429070163678887
Number of clusters: 3, Silhouette Score: 0.47909095516615025
Number of clusters: 4, Silhouette Score: 0.44480350310619
Number of clusters: 5, Silhouette Score: 0.3726839061164574
Number of clusters: 6, Silhouette Score: 0.3647760319184738
Number of clusters: 7, Silhouette Score: 0.36934013776268393
Number of clusters: 8, Silhouette Score: 0.3593656519686193
Number of clusters: 9, Silhouette Score: 0.3610747142201551
Best number of clusters by silhouette score: 3


/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/opt/anaconda3/envs/cma/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 1

In [25]:
import pandas as pd
import numpy as np
from tslearn.metrics import dtw
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

shorest_traj = 9

df = df_total

# record the raw data
ID = df.iloc[0]['ID']
trajectory = 1
original_series = []
traj = []

for i in range(df.shape[0]):
    if df.iloc[i]['ID'] != ID and traj:
        if len(traj) >= shorest_traj:
            original_series.append(traj)
        ID = df.iloc[i]['ID']
        traj = []
    else:
        if df.iloc[i]['Trajectory'] != trajectory and traj:
            if len(traj) >= shorest_traj:
                original_series.append(traj)
            trajectory = df.iloc[i]['Trajectory']
            traj = []
    traj.append([df.iloc[i]['Speed Change'], df.iloc[i]['Direction Change'], df.iloc[i]['Speed']])
original_series.append(traj)

df.pop('Speed')

#normalization

df['Speed Change'] = 2 * (df['Speed Change'] - df['Speed Change'].min()) / (df['Speed Change'].max() - df['Speed Change'].min()) - 1
df['Direction Change'] = 2 * (df['Direction Change'] - df['Direction Change'].min()) / (df['Direction Change'].max() - df['Direction Change'].min()) - 1

# cluster based on normalized data
ID = df.iloc[0]['ID']
trajectory = 1
series = []
traj = []
for i in range(df.shape[0]):
    if df.iloc[i]['ID'] != ID and traj:
        if len(traj) >= shorest_traj + 1:
            series.append(traj)
        ID = df.iloc[i]['ID']
        traj = []
    else:
        if df.iloc[i]['Trajectory'] != trajectory and traj:
            if len(traj) >= shorest_traj + 1:
                series.append(traj)
            trajectory = df.iloc[i]['Trajectory']
            traj = []
    if not traj:
        traj.append([ID, trajectory])
    traj.append([df.iloc[i]['Speed Change'], df.iloc[i]['Direction Change']])
series.append(traj)

time_series_data = [np.array(ts) for ts in series]

first_columns = [ts[0] for ts in time_series_data]  # Store the first column
processed_series = [ts[1:] for ts in time_series_data]
# print(processed_series[879])

# Step 2: Compute the pairwise DTW distance matrix using only the second column
n = len(processed_series)
dtw_distances = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        distance = dtw(processed_series[i], processed_series[j])
        dtw_distances[i, j] = distance
        dtw_distances[j, i] = distance

# Convert the distance matrix to a condensed distance matrix format
dtw_distances_condensed = squareform(dtw_distances)

# Step 3: Perform hierarchical clustering using the precomputed distance matrix
Z = linkage(dtw_distances_condensed, method='ward')

# Step 4: Optional: Visualize the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(Z, no_labels=True)
plt.title("Dendrogram of Hierarchical Clustering with DTW")
plt.ylabel("Distance")
plt.savefig('Dendrogram.png')
plt.show()

# Step 5: Determine the optimal number of clusters using the Silhouette score
range_n_clusters = range(2, 10)  # Example range to check
best_n_clusters = None
best_silhouette_score = -1

for n_clusters in range_n_clusters:
    cluster_labels = fcluster(Z, t=n_clusters, criterion='maxclust')
    silhouette_avg = silhouette_score(dtw_distances, cluster_labels, metric='precomputed')
    print(f"For n_clusters = {n_clusters}, the Silhouette score is {silhouette_avg:.4f}")

    if silhouette_avg > best_silhouette_score:
        best_silhouette_score = silhouette_avg
        best_n_clusters = n_clusters

print(f"\nOptimal number of clusters: {best_n_clusters} with a Silhouette score of {best_silhouette_score:.4f}")

# Step 6: Use the optimal number of clusters to finalize clustering
cluster_labels = fcluster(Z, t= best_n_clusters, criterion='maxclust')

# Step 7: Concatenate the first column back to the clustered data


clustered_dataset = []

for i in range(n):
    length = len(original_series[i])
    clustered_data = pd.DataFrame({
        'ID':[first_columns[i][0]] * length,
        'Trajectory':[first_columns[i][1]] * length,
        'Speed Change':[change[0] for change in original_series[i]],
        'Direction Change':[change[1] for change in original_series[i]],
        'Speed':[change[2] for change in original_series[i]],
        'Cluster':[cluster_labels[i]] * length
    })
    clustered_dataset.append(clustered_data)

clustered_dataset = pd.concat(clustered_dataset, ignore_index=True)

clustered_dataset.to_csv('./clustered.csv', index = False)

cluster_2 = 0
for i in range(len(cluster_labels)):
    if cluster_labels[i] == 2:
        cluster_2+=1
print(cluster_2, len(cluster_labels))



KeyError: 'Speed'